Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Studying the feature vectors in the feature space

In [1]:
import os
from six.moves import xrange  
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io
import warnings
import scipy
from scipy import stats 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112
cluster_length=16
feature_size=8192 
saved_path='/nobackup/leopauly/S2l/MIME/90_10_shuffle/'

####  Loading model

In [3]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, cluster_length,height,width,channel],name='x') 
model_keras = md.C3D_MIME20_training_model_tf(summary=True)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)

print('Miscellenious items finished..!!',flush=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

In [4]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-155'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/MIME/90_10_shuffle/activity_model.ckpt-155
Model restored from file: /nobackup/leopauly/S2l/MIME/90_10_shuffle/


#### wessertein distance

In [5]:
def _validate_distribution(values, weights):
    """
    Validate the values and weights from a distribution input of `cdf_distance`
    and return them as ndarray objects.
    Parameters
    ----------
    values : array_like
        Values observed in the (empirical) distribution.
    weights : array_like
        Weight for each value.
    Returns
    -------
    values : ndarray
        Values as ndarray.
    weights : ndarray
        Weights as ndarray.
    """
    # Validate the value array.
    values = np.asarray(values, dtype=float)
    if len(values) == 0:
        raise ValueError("Distribution can't be empty.")

    # Validate the weight array, if specified.
    if weights is not None:
        weights = np.asarray(weights, dtype=float)
        if len(weights) != len(values):
            raise ValueError('Value and weight array-likes for the same '
                             'empirical distribution must be of the same size.')
        if np.any(weights < 0):
            raise ValueError('All weights must be non-negative.')
        if not 0 < np.sum(weights) < np.inf:
            raise ValueError('Weight array-like sum must be positive and '
                             'finite. Set as None for an equal distribution of '
                             'weight.')

        return values, weights

    return values, None


In [6]:
def _cdf_distance(p, u_values, v_values, u_weights=None, v_weights=None):
    r"""
    Compute, between two one-dimensional distributions :math:`u` and
    :math:`v`, whose respective CDFs are :math:`U` and :math:`V`, the
    statistical distance that is defined as:
    .. math::
        l_p(u, v) = \left( \int_{-\infty}^{+\infty} |U-V|^p \right)^{1/p}
    p is a positive parameter; p = 1 gives the Wasserstein distance, p = 2
    gives the energy distance.
    Parameters
    ----------
    u_values, v_values : array_like
        Values observed in the (empirical) distribution.
    u_weights, v_weights : array_like, optional
        Weight for each value. If unspecified, each value is assigned the same
        weight.
        `u_weights` (resp. `v_weights`) must have the same length as
        `u_values` (resp. `v_values`). If the weight sum differs from 1, it
        must still be positive and finite so that the weights can be normalized
        to sum to 1.
    Returns
    -------
    distance : float
        The computed distance between the distributions.
    Notes
    -----
    The input distributions can be empirical, therefore coming from samples
    whose values are effectively inputs of the function, or they can be seen as
    generalized functions, in which case they are weighted sums of Dirac delta
    functions located at the specified values.
    References
    ----------
    .. [1] Bellemare, Danihelka, Dabney, Mohamed, Lakshminarayanan, Hoyer,
           Munos "The Cramer Distance as a Solution to Biased Wasserstein
           Gradients" (2017). :arXiv:`1705.10743`.
    """
    u_values, u_weights = _validate_distribution(u_values, u_weights)
    v_values, v_weights = _validate_distribution(v_values, v_weights)

    u_sorter = np.argsort(u_values)
    v_sorter = np.argsort(v_values)

    all_values = np.concatenate((u_values, v_values))
    all_values.sort(kind='mergesort')

    # Compute the differences between pairs of successive values of u and v.
    deltas = np.diff(all_values)

    # Get the respective positions of the values of u and v among the values of
    # both distributions.
    u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
    v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

    # Calculate the CDFs of u and v using their weights, if specified.
    if u_weights is None:
        u_cdf = u_cdf_indices / u_values.size
    else:
        u_sorted_cumweights = np.concatenate(([0],
                                              np.cumsum(u_weights[u_sorter])))
        u_cdf = u_sorted_cumweights[u_cdf_indices] / u_sorted_cumweights[-1]

    if v_weights is None:
        v_cdf = v_cdf_indices / v_values.size
    else:
        v_sorted_cumweights = np.concatenate(([0],
                                              np.cumsum(v_weights[v_sorter])))
        v_cdf = v_sorted_cumweights[v_cdf_indices] / v_sorted_cumweights[-1]
        
    
    # Compute the value of the integral based on the CDFs.
    # If p = 1 or p = 2, we avoid using np.power, which introduces an overhead
    # of about 15%.
    if p == 1:
        return np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))
    if p == 2:
        return np.sqrt(np.sum(np.multiply(np.square(u_cdf - v_cdf), deltas)))
    return np.power(np.sum(np.multiply(np.power(np.abs(u_cdf - v_cdf), p),
                                       deltas)), 1/p)


In [7]:

def wasserstein_distance(u_values, v_values, u_weights=None, v_weights=None):
    r"""
    Compute the first Wasserstein distance between two 1D distributions.
    This distance is also known as the earth mover's distance, since it can be
    seen as the minimum amount of "work" required to transform :math:`u` into
    :math:`v`, where "work" is measured as the amount of distribution weight
    that must be moved, multiplied by the distance it has to be moved.
    .. versionadded:: 1.0.0
    Parameters
    ----------
    u_values, v_values : array_like
        Values observed in the (empirical) distribution.
    u_weights, v_weights : array_like, optional
        Weight for each value. If unspecified, each value is assigned the same
        weight.
        `u_weights` (resp. `v_weights`) must have the same length as
        `u_values` (resp. `v_values`). If the weight sum differs from 1, it
        must still be positive and finite so that the weights can be normalized
        to sum to 1.
    Returns
    -------
    distance : float
        The computed distance between the distributions.
    Notes
    -----
    The first Wasserstein distance between the distributions :math:`u` and
    :math:`v` is:
    .. math::
        l_1 (u, v) = \inf_{\pi \in \Gamma (u, v)} \int_{\mathbb{R} \times
        \mathbb{R}} |x-y| \mathrm{d} \pi (x, y)
    where :math:`\Gamma (u, v)` is the set of (probability) distributions on
    :math:`\mathbb{R} \times \mathbb{R}` whose marginals are :math:`u` and
    :math:`v` on the first and second factors respectively.
    If :math:`U` and :math:`V` are the respective CDFs of :math:`u` and
    :math:`v`, this distance also equals to:
    .. math::
        l_1(u, v) = \int_{-\infty}^{+\infty} |U-V|
    See [2]_ for a proof of the equivalence of both definitions.
    The input distributions can be empirical, therefore coming from samples
    whose values are effectively inputs of the function, or they can be seen as
    generalized functions, in which case they are weighted sums of Dirac delta
    functions located at the specified values.
    References
    ----------
    .. [1] "Wasserstein metric", https://en.wikipedia.org/wiki/Wasserstein_metric
    .. [2] Ramdas, Garcia, Cuturi "On Wasserstein Two Sample Testing and Related
           Families of Nonparametric Tests" (2015). :arXiv:`1509.02237`.
    Examples
    --------
    >>> from scipy.stats import wasserstein_distance
    >>> wasserstein_distance([0, 1, 3], [5, 6, 8])
    5.0
    >>> wasserstein_distance([0, 1], [0, 1], [3, 1], [2, 2])
    0.25
    >>> wasserstein_distance([3.4, 3.9, 7.5, 7.8], [4.5, 1.4],
    ...                      [1.4, 0.9, 3.1, 7.2], [3.2, 3.5])
    4.0781331438047861
    """
    return _cdf_distance(1, u_values, v_values, u_weights, v_weights)

####  Feature extraction and Distance calculation

In [8]:
## Uniform sampling of frames
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):

    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr) #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [9]:
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name('flatten_1/Reshape:0') 
    #'flatten_1/Reshape:0' #dropout_1/cond/Merge:0 #fc8/BiasAdd:0 #Softmax:0
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

In [10]:
## Feature extraction
def get_features_from_class(class_folder):
    feature_set_a=[]
    base_dir_a=class_folder
    sub_dir_a=os.listdir(base_dir_a)
    sub_dir_a=sorted(sub_dir_a)
    print(sub_dir_a)
    if '.DS_Store' in sub_dir_a:
        sub_dir_a.remove('.DS_Store')
    for sub_dir_a_ in sub_dir_a:
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        feature_set_a.append(extract_video_features(vid_a))
    return np.array(feature_set_a)

In [11]:
feature_set_a=get_features_from_class('/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/')
feature_set_b=get_features_from_class('/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/')
feature_set_c=get_features_from_class('/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/')

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']
['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_ne

In [12]:
scaler = MinMaxScaler()
scaler.fit(np.concatenate((feature_set_a,feature_set_b,feature_set_c),axis=0))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
def dist_calc(value_a,value_b):
    
    

    value_a=scaler.transform(value_a)
    value_b=scaler.transform(value_b)
    distance_=wasserstein_distance(range(len(value_a)), range(len(value_b)), value_a, value_b)
    #distance_=wasserstein_distance(value_a, value_b)
    
    
    return(distance_)

------------------------------------------------------------------------------------------------------------------

#### Reach

In [14]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('\nInter class variance bw class :',intra_var_class)
print('Number of combinations:',np.shape(dist_sim_class_nozero))

intra_reach=intra_var_class

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Inter class variance bw class : 177.336821946
Number of combinations: (272,)


In [15]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter1_reach=inter_var

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class 1 and class 2: 205.657645084
Number of combinations: (289,)


In [16]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter2_reach=inter_var

Sub directories: ['Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Inter class variance bw class 1 and class 2: 146.200075739
Number of combinations: (289,)


------------------------------------------------------------------------------------------------------------------

#### Push

In [17]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter1_push=inter_var

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class 1 and class 2: 205.657645084
Number of combinations: (289,)


In [18]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('\nInter class variance bw class :',intra_var_class)
print('Number of combinations:',np.shape(dist_sim_class_nozero))

intra_push=intra_var_class

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class : 240.947783098
Number of combinations: (272,)


In [19]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter2_push=inter_var

Sub directories: ['Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class 1 and class 2: 187.750899496
Number of combinations: (289,)


------------------------------------------------------------------------------------------------------------------

#### Push n Reach 

In [20]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

inter1_rnp=inter_var

['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']
['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_change_pos', 'reach_robo_change_target', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_comp2', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
Inter class variance bw class 1 and class 2: 146.200075739
Shape (289,)


In [21]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

inter2_rnp=inter_var

['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']
['.DS_Store', 'push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_comp2', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']
Inter class variance bw class 1 and class 2: 187.750899496
Shape (289,)


In [22]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Leeds_Action_Dataset_6_7_2020/Reach n Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('Inter class variance bw class :',intra_var_class)


intra_rnp=intra_var_class

['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']
['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_comp2', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']
Inter class variance bw class : 112.5912146


------------------------------------------------------------------------------------------------------------------

In [23]:
print(round(intra_reach,4))
print(round(inter1_reach,4))
print(round(inter2_reach,4))


print('      ',round(inter1_push,4))
print('      ',round(intra_push,4))
print('      ',round(inter2_push,4))


print('      ','      ',round(inter1_rnp,4))
print('      ','      ',round(inter2_rnp,4))
print('      ','      ',round(intra_rnp,4))


177.3368
205.6576
146.2001
       205.6576
       240.9478
       187.7509
              146.2001
              187.7509
              112.5912
